In [1]:
import keras
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from keras.layers import Dense, Activation,MaxPool1D, MaxPool2D, Conv1D, Conv2D, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D
from keras.models import Sequential, Model, model_from_json
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from sklearn.utils import shuffle
from keras import applications
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from sklearn.model_selection import train_test_split
import cv2
%matplotlib inline

Using TensorFlow backend.


In [2]:
EXTRACT_FEATURES = False
FEATURE_FILE_NAME = "inception_features.npy"
np.random.seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [3]:
def load(fname,test=False, cols=None):
    df = pd.read_csv(fname)
    
    df['Image'] = df['Image'].apply(lambda im: np.fromstring(im, sep=' '))
    
    if cols:
        df = df[list(cols)+['Image']]
    
    #print( df.count())
    df = df.dropna()
    columns = df.columns
    
    X = np.vstack(df['Image'].values)#/255
    X = X.astype(np.float32)
    
    if not test:
        y = df[df.columns[:-1]].values
        #y = (y-48)/48
        X, y = shuffle(X, y, random_state=20)
        y = y.astype(np.float32)
    else:
        y = None
        columns = None
    
    return X, y, columns

def load2d(fname,test=False, cols=None):
    
    X, y, columns = load(fname,test, cols)
    X = X.reshape(-1,96,96, 1)
    
    return X, y, columns

def array_to_img(x):
#     rgbimg = cv2.resize(cv2.cvtColor(x,cv2.COLOR_GRAY2RGB),(224,224))
    rgbimg = cv2.cvtColor(x,cv2.COLOR_GRAY2RGB)
    rgbimg = rgbimg[...,::-1].astype(np.float32)
    a = np.expand_dims(rgbimg, axis=0)
    a = preprocess_input(a)
    return a

In [4]:
X, y, columns = load2d("data/training.csv",test=False)
columns = np.array(list(columns[:-1]))
columns
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [5]:
inception_model = applications.inception_v3.InceptionV3(include_top=False, weights='imagenet', pooling='avg')

In [20]:
inception_features = []
inception_features_array = np.zeros([X_train.shape[0],2048])

if EXTRACT_FEATURES:
    for i in range(X_train.shape[0]):
        if(i%10 == 0):
            print('.',end='')
        img = array_to_img(X_train[i])
        features = inception_model.predict(img)
#         print(features.shape)
        inception_features_array[i] = features.flatten()
    file = open(FEATURE_FILE_NAME,"wb")
#     inception_features_array = np.array(inception_features)
    np.save(file,inception_features_array)
else:
    file = open(FEATURE_FILE_NAME,'rb')
    inception_features_array = np.load(file)

In [21]:
model = Sequential()
model.add(BatchNormalization())

model.add(Conv1D(32,3,strides=1))
model.add(Activation("relu"))
model.add(MaxPool1D(pool_size=2,strides=2))
#model.add(Dropout(0.15))

model.add(Conv1D(64,2,strides=1))
model.add(Activation("relu"))
model.add(MaxPool1D(pool_size=2,strides=2))
#model.add(Dropout(0.15))

model.add(Conv1D(128,2,strides=1))
model.add(Activation("relu"))
model.add(MaxPool1D(pool_size=2,strides=2))
#model.add(Dropout(0.15))

model.add(Flatten())
model.add(Dense(500))
model.add(Activation("relu"))

model.add(Dense(30))
model.add(Activation("relu"))

In [22]:
model.compile(loss="mse", optimizer="rmsprop",metrics=["accuracy"])

In [23]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling build() or calling fit() with some data. Or specify input_shape or batch_input_shape in the first layer for automatic build. 

In [25]:
# save best weights
checkpointer = ModelCheckpoint(filepath='face_model_in1_cnn.h5', verbose=1, save_best_only=True)

# num epochs
epochs = 10
model.load_weights("face_model_in1_cnn.h5")

# X_train, X_test, y_train, y_test = train_test_split(inception_features_array, y, test_size=0.15, random_state=42)
# run model
inception_features_array = np.expand_dims(inception_features_array, axis=2)
hist = model.fit(inception_features_array,y_train,batch_size=1,epochs=epochs, 
                 validation_split=0.15, callbacks=[checkpointer], verbose=2)

# save model to json
model_json = model.to_json()
with open("face_model_in1_cnn.json", "w") as json_file:
    json_file.write(model_json)

Train on 1546 samples, validate on 273 samples
Epoch 1/10
 - 311s - loss: 187.4077 - acc: 0.7044 - val_loss: 191.6057 - val_acc: 0.6923

Epoch 00001: val_loss improved from inf to 191.60572, saving model to face_model_in1_cnn.h5
Epoch 2/10
 - 311s - loss: 186.9652 - acc: 0.6908 - val_loss: 189.9089 - val_acc: 0.6777

Epoch 00002: val_loss improved from 191.60572 to 189.90895, saving model to face_model_in1_cnn.h5
Epoch 3/10
 - 311s - loss: 186.6950 - acc: 0.7025 - val_loss: 188.9948 - val_acc: 0.6264

Epoch 00003: val_loss improved from 189.90895 to 188.99475, saving model to face_model_in1_cnn.h5
Epoch 4/10
 - 311s - loss: 186.2976 - acc: 0.6966 - val_loss: 195.7082 - val_acc: 0.6960

Epoch 00004: val_loss did not improve from 188.99475
Epoch 5/10
 - 324s - loss: 185.9715 - acc: 0.7122 - val_loss: 191.3685 - val_acc: 0.6996

Epoch 00005: val_loss did not improve from 188.99475
Epoch 6/10
 - 339s - loss: 185.6777 - acc: 0.7076 - val_loss: 189.8553 - val_acc: 0.6996

Epoch 00006: val_lo

In [26]:
#model.load_weights("face_model_in1_cnn.h5")
inception_features_array_test = np.zeros([X_test.shape[0],2048])
for i in range(X_test.shape[0]):
    img = array_to_img(X_test[i])
    features = inception_model.predict(img)
    inception_features_array_test[i] = features.flatten()
inception_features_array_test = np.expand_dims(inception_features_array_test,axis=2)
score, acc = model.evaluate(inception_features_array_test,y_test)
print(score, acc)

321/321 [==============================] - 0s 1ms/step
194.1385484737028 0.6915887850467289
